In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import csv

import numpy as np
import sys
import io
import time
import pandas as pd
import numpy as np
from random import randint
import os
import itertools

import random
import pickle
from io import BytesIO
import json
import time

In [ ]:
# helper function for movement
import numpy as np
from random import randint

def vector_add(a, b):
  x = a[0] + b[0]
  y = a[1] + b[1]
  return (x,y)

generation_score = []

In [ ]:
import time

class Game:
  def __init__(self):
    self.grid = np.zeros((20,20))
    self.pos = [(10,10)]
    self.speed = 1
    self.direction = 0
    self.length = 1
    self.apple_pos = (5,5)
    self.playing = True
    self.since_ate = 0

  def update(self):
    self.since_ate += 1
    for i in range(self.length-1,0,-1):
        self.pos[i] = self.pos[i-1]

    if self.direction == 0:
        self.pos[0] = vector_add(self.pos[0], (1, 0))
    if self.direction == 1:
        self.pos[0] = vector_add(self.pos[0], (-1, 0))
    if self.direction == 2:
        self.pos[0] = vector_add(self.pos[0], (0, 1))
    if self.direction == 3:
        self.pos[0] = vector_add(self.pos[0], (0, -1))
    
    # check to see for collision with self

    self.playing = not self.get_crashed()

    if self.ateApple(self.apple_pos[0], self.apple_pos[1]):
      # print('ate apple')
      self.length += 1
      self.pos.append((0,0))
      self.apple_pos = self.generate_apple_pos()
      self.since_ate = 0

    # time.sleep(.00001)

  def moveRight(self):
    if vector_add(self.pos[0], (1, 0)) not in self.pos:
      self.direction = 0

  def moveLeft(self):
    if vector_add(self.pos[0], (-1, 0)) not in self.pos:
      self.direction = 1

  def moveUp(self):
    if vector_add(self.pos[0], (0, 1)) not in self.pos:
      self.direction = 2

  def moveDown(self):
    if vector_add(self.pos[0], (0, -1)) not in self.pos:
      self.direction = 3

  def get_crashed(self):
    x, y = self.pos[0]
    outside = (x < 0 or x >= 20 or y < 0 or y >= 20)
    inside = self.isCollision(self.pos[0][0], self.pos[0][1])
    length = self.length > 361
    return outside or inside or length
  
  def get_playing(self):
    return not self.get_crashed(self)

  def isCollision(self,x1,y1):
    return (x1, y1) in self.pos[1:]
  
  def ateApple(self, x1, y1):
    return (x1, y1) in self.pos

  def get_apple_pos(self):
    return self.apple_pos

  def generate_apple_pos(self):
    generate = True
    while(generate):
      x = randint(1,19)
      y = randint(1,19)

      if (x, y) not in self.pos:
        return (x, y)

  def get_length(self):
    return self.length

  def do_nothing(self):
    self.direction = self.direction
  
  def restart(self):
    # print("restarting")
    self.pos = [(10,10)]
    self.speed = 1
    self.direction = 0
    self.length = 1
    self.apple_pos = (5,5)
    self.playing = True
    self.since_ate = 0

  def get_grid(self):
    grid = np.zeros((20, 20))
    for i in range(len(self.pos)):
      x, y = self.pos[i]
      grid[x, y] = 1
    x, y = self.apple_pos
    grid[x,y] = 100
    return grid

  def time_since_ate(self):
    return self.since_ate

In [ ]:
game = Game()
# print(game.apple_pos)
# print(game.pos)
# for i in range(20):
#   game.moveDown()
#   game.update()
#   game.moveLeft()
#   game.update()
# print(game.pos)
# print(game.get_crashed())
# print(game.playing)
# print(game.length)
# print(game.apple_pos)

# cont = True

# while(cont):
#   print(game.pos)
#   print(game.apple_pos)
#   print(game.since_ate)
#   a = input("input move: ")
#   if int(a) in [0,1,2,3,4]:
#     a = int(a)
#     print(a)
#     if a == 0:
#       game.moveLeft()
#       game.update()
#     elif a == 1:
#       game.moveRight()
#       game.update()
#     elif a == 2:
#       game.moveDown()
#       game.update()
#     elif a == 3:
#       game.moveUp()
#       game.update()
#     elif a == 4:
#       game.update()
#   if a == 'end':
#     cont = False

In [ ]:
class SnakeAgent:
  def __init__(self, game):
    self.snakeGame = game
  def is_running(self):
    return self.snakeGame.get_playing()
  def is_crashed(self):
    return self.snakeGame.get_crashed()
  def left(self):
    self.snakeGame.moveLeft()
  def right(self):
    self.snakeGame.moveRight()
  def down(self):
    self.snakeGame.moveDown()
  def up(self):
    self.snakeGame.moveUp()
  def DoNothing(self):
    self.snakeGame.do_nothing()

In [ ]:
class Game_State:
  def __init__(self, agent, game):
    self._agent = agent
    self.snakeGame = game
  def get_next_state(self, actions):
    score = self.snakeGame.get_length()

    reward = self.get_reward()
    is_over = False

    if actions[0] == 1:
      self._agent.left()
    elif actions[1] == 1:
      self._agent.right()
    elif actions[2] == 1:
      self._agent.up()
    elif actions[3] == 1:
      self._agent.down()
    elif actions[4] == 1:
      self._agent.DoNothing()

    self.snakeGame.update()

    if self._agent.is_crashed():
      generation_score.append(score)
      # print("restart")
      time.sleep(.0001)
      self.snakeGame.restart()
      reward = -1
      is_over = True

    grid = self.snakeGame.get_grid()
    grid = np.reshape(grid, (20,20,1))
    grid = np.transpose(grid, (2,0,1))
    tensor = torch.from_numpy(grid).type(torch.FloatTensor)
    if torch.cuda.is_available():
      tensor = tensor.cuda()

    return tensor, reward, is_over, score
  def get_reward(self):
    # base = .00001+self.snakeGame.get_length()-1
    length = self.snakeGame.get_length()
    base = length*(length+1)/2 - 1
    time = self.snakeGame.time_since_ate()
    if time >= 10:
      base -= time*.0001
    
    return base

In [ ]:
game = Game()
agent = SnakeAgent(game)
state = Game_State(agent, game)
actions = [0,0,0,0,1]
state.get_next_state(actions)
state.get_next_state(actions)
actions = [0,0,0,1,0]
state.get_next_state(actions)

for i in range(4):
  state.get_next_state(actions)

actions = [1,0,0,0,0]

for i in range(6):
  state.get_next_state(actions)

state.get_next_state(actions)

state.get_next_state(actions)
a, b, c, d = state.get_next_state(actions)
print(a.shape)


torch.Size([1, 20, 20])


In [1]:
class NeuralNetwork(nn.Module):

    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.number_of_actions = 5
        self.gamma = 0.95
        self.final_epsilon = 0.0001
        self.initial_epsilon = 0.2
        self.number_of_iterations = 500001
        self.replay_memory_size = 10000
        self.minibatch_size = 32
        
        self.conv1 = nn.Conv2d(4, 32, 8, 2)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(32, 64, 4, 2)
        self.relu2 = nn.ReLU(inplace=True)
        # self.fc4 = nn.Linear(256, 512)
        # self.relu4 = nn.ReLU(inplace=True)
        self.fc5 = nn.Linear(512 , self.number_of_actions)

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        # out = self.conv3(out)
        # out = self.relu3(out)
        out = out.view(out.size()[0], -1)
        # out = self.fc4(out)
        # out = self.relu4(out)
        out = self.fc5(out)

        return out


def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.uniform(m.weight, -0.01, 0.01)
        m.bias.data.fill_(0.01)

def train(model, start):
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.MSELoss()

    game = Game()
    snake = SnakeAgent(game)
    game_state = Game_State(snake,game)

    replay_memory = []

    action = torch.zeros([model.number_of_actions], dtype=torch.float32)
    action[0] = 1
    print(action)
    image_data, reward, terminal, score = game_state.get_next_state(action)
    print(image_data.shape)

    state = torch.cat((image_data, image_data, image_data, image_data)).unsqueeze(0) #stacking 4 images

    print("printing size of input state at 0")
    print(state.size())

    epsilon = model.initial_epsilon
    iteration = 0

    epsilon_decrements = np.linspace(model.initial_epsilon, model.final_epsilon,model.number_of_iterations)

    q_vals = []

    while iteration < model.number_of_iterations:
        output = model(state)[0]

        action = torch.zeros([model.number_of_actions], dtype=torch.float32)
        if torch.cuda.is_available():
            action = action.cuda()

        random_action = random.random() <= epsilon
        action_index = [torch.randint(model.number_of_actions, torch.Size([]), dtype=torch.int)
                        if random_action
                        else torch.argmax(output)][0]

        if torch.cuda.is_available():
            action_index = action_index.cuda()

        action[action_index] = 1

        image_data_1, reward, terminal, score = game_state.get_next_state(action)
        state_1 = torch.cat((state.squeeze(0)[1:, :, :], image_data_1)).unsqueeze(0)

        action = action.unsqueeze(0)
        reward = torch.from_numpy(np.array([reward], dtype=np.float32)).unsqueeze(0)

        replay_memory.append((state, action, reward, state_1, terminal))

        if len(replay_memory) > model.replay_memory_size:
            replay_memory.pop(0)

        epsilon = epsilon_decrements[iteration]
        minibatch = random.sample(replay_memory, min(len(replay_memory), model.minibatch_size))

        state_batch = torch.cat(tuple(d[0] for d in minibatch))
        action_batch = torch.cat(tuple(d[1] for d in minibatch))
        reward_batch = torch.cat(tuple(d[2] for d in minibatch))
        state_1_batch = torch.cat(tuple(d[3] for d in minibatch))

        if torch.cuda.is_available():
            state_batch = state_batch.cuda()
            action_batch = action_batch.cuda()
            reward_batch = reward_batch.cuda()
            state_1_batch = state_1_batch.cuda()

        output_1_batch = model(state_1_batch)

        y_batch = torch.cat(tuple(reward_batch[i] if minibatch[i][4]
                                  else reward_batch[i] + model.gamma * torch.max(output_1_batch[i])
                                  for i in range(len(minibatch))))

        q_value = torch.sum(model(state_batch) * action_batch, dim=1)
        optimizer.zero_grad()

        y_batch = y_batch.detach()
        loss = criterion(q_value, y_batch)

        loss.backward()
        optimizer.step()

        state = state_1

        global generation_score
        if len(generation_score) == 0:
            avg_score = 0
        else:
            avg_score = sum(generation_score)/len(generation_score)

        if iteration % 1000 == 0:
            print("iteration:", iteration, "score:", score, "elapsed time:", time.time() - start, "epsilon:", epsilon, "action:",
                  action_index.cpu().detach().numpy(), "reward:", reward.numpy()[0][0], "Q max:",
                  np.max(output.cpu().detach().numpy()), "avg_score:", avg_score)
            q_vals.append(np.max(output.cpu().detach().numpy()))
            generation_score = []

        if iteration % 100000 == 0:
            torch.save(model, "pretrained-model/current_model_" + str(iteration) + ".pth")

        iteration += 1
    return q_vals

def test(model):

    game = Game()
    snake = SnakeAgent(game)
    game_state = Game_State(snake,game)


    action = torch.zeros([model.number_of_actions], dtype=torch.float32)
    action[0] = 1
    image_data, reward, terminal, s_, = game_state.get_next_state(action)
    state = torch.cat((image_data, image_data, image_data, image_data)).unsqueeze(0)

    high_score = 0
    tot_counter = 0
    cur_counter = 0

    while True:
        output = model(state)[0]

        action = torch.zeros([model.number_of_actions], dtype=torch.float32)
        if torch.cuda.is_available():
            action = action.cuda()

        action_index = torch.argmax(output)
        if torch.cuda.is_available():
            action_index = action_index.cuda()
        action[action_index] = 1

        image_data_1, reward, terminal, s_ = game_state.get_next_state(action)
        state_1 = torch.cat((state.squeeze(0)[1:, :, :], image_data_1)).unsqueeze(0)

        # print(action)
        if cur_counter >= 10000:
          terminal = True

        if terminal == True:
          print("restarting")
          tot_counter += 1
          if s_ > high_score:
            high_score = s_
            print(s_)
          if tot_counter >= 200:
            print(high_score)
            return high_score
        # time.sleep(.5)
        # print()

        cur_counter += 1
        state = state_1

def main(mode):
    cuda_is_available = torch.cuda.is_available()

    if mode == 'test':

        model = torch.load(
            'pretrained-model/current_model_500000.pth',
            map_location='cpu' if not cuda_is_available else None
        ).eval()

        print("loaded")

        if cuda_is_available:
            model = model.cuda()

        test(model)

    elif mode == 'train':
        if not os.path.exists('pretrained-model/'):
            os.mkdir('pretrained-model/')

        model = NeuralNetwork()

        if cuda_is_available:
            model = model.cuda()

        model.apply(init_weights)
        start = time.time()

        vals = train(model, start)
        with open('q_vals.csv', 'w', newline='') as myfile:
          wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
          wr.writerow(vals)


if __name__ == "__main__":
    main('train')


NameError: ignored